<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Interacting with Queues in Storage

## Azure Actions Covered

* Creating queues in a storage container
* Sending and receiving messages from a queue
* Deleting queues

In this lecture, we're going to take a look at how to interact with queues in storage containers via the Python SDK.

To begin, we import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). 

In [1]:
from azure.identity import AzureCliCredential
# New imports for queues
from azure.storage.queue import QueueClient, QueueServiceClient, QueueMessage

from settings import AZURE_SUBSCRIPTION_ID

Make sure to log in to the Azure CLI and then get the relevant credential.

In [2]:
credential = AzureCliCredential()

We can instantiate our `QueueClient()` to create and interact with our queues. It needs the following parameters:
* `account_url` - URL for the queue in the storage account. Will be of the form `https://<storage-account>.queue.core.windows.net`
* `queue_name` - Name for the queue to be created or interacted with
* `credential` - Azure credential to use for authentication.

In [3]:
queue_client = QueueClient(
    account_url='https://benbstorage1234.queue.core.windows.net',
    queue_name='bens-queue-1',
    credential=credential
)

We can call the `create_queue` method to create our new queue.

In [7]:
queue = queue_client.create_queue()

Now that a queue has been created, we can use our `QueueClient`to send messages to the queue.

In [ ]:
queue_client.send_message("My first message")

In [21]:
queue_client.send_message("My second message")

{'id': '31ac7778-d5c9-4c89-81c7-fac644a4d20c', 'inserted_on': datetime.datetime(2023, 5, 15, 5, 18, 9, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2023, 5, 22, 5, 18, 9, tzinfo=datetime.timezone.utc), 'dequeue_count': None, 'content': 'My second message', 'pop_receipt': 'AgAAAAMAAAAAAAAAwOQso+yG2QE=', 'next_visible_on': datetime.datetime(2023, 5, 15, 5, 18, 9, tzinfo=datetime.timezone.utc)}

You can see some of the messages in the queue using `peek_messages()`.

In [22]:
queue_client.peek_messages(max_messages=2)

[{'id': '2f8cbc3a-ae75-406c-acb1-7a084c1fc07a', 'inserted_on': datetime.datetime(2023, 5, 15, 5, 10, 31, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2023, 5, 22, 5, 10, 31, tzinfo=datetime.timezone.utc), 'dequeue_count': 1, 'content': 'My first message', 'pop_receipt': None, 'next_visible_on': None},
 {'id': '5b0b3330-8e67-416a-9e5a-5ba9cea9f79b', 'inserted_on': datetime.datetime(2023, 5, 15, 5, 18, 7, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2023, 5, 22, 5, 18, 7, tzinfo=datetime.timezone.utc), 'dequeue_count': 0, 'content': 'My first message', 'pop_receipt': None, 'next_visible_on': None}]

To retrieve the message at the front of the queue and remove it from the queue (i.e. dequeue it), we can use `receive_message()` and assign the result to a variable in order to interact with the message.

In [23]:
first_message = queue_client.receive_message()

In [24]:
first_message

{'id': '2f8cbc3a-ae75-406c-acb1-7a084c1fc07a', 'inserted_on': datetime.datetime(2023, 5, 15, 5, 10, 31, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2023, 5, 22, 5, 10, 31, tzinfo=datetime.timezone.utc), 'dequeue_count': 2, 'content': 'My first message', 'pop_receipt': 'AgAAAAMAAAAAAAAArZwWuOyG2QE=', 'next_visible_on': datetime.datetime(2023, 5, 15, 5, 18, 44, tzinfo=datetime.timezone.utc)}

You can update messages in a queue as well using `update_message()`. For this, you do need:
* `message` - The message object or ID to identify the message to update.
* `pop_receipt` - Pop receipt value from earlier call to the message
* `content` - Content to put in updated message.

In [25]:
queue_client.update_message(
    message=first_message.id,
    pop_receipt=first_message.pop_receipt,
    content="Was this my first message?"
)

{'id': '2f8cbc3a-ae75-406c-acb1-7a084c1fc07a', 'inserted_on': None, 'expires_on': None, 'dequeue_count': None, 'content': 'Was this my first message?', 'pop_receipt': 'AwAAAAMAAAAAAAAAgvLRsOyG2QECAAAA', 'next_visible_on': datetime.datetime(2023, 5, 15, 5, 18, 32, tzinfo=datetime.timezone.utc)}

To get some of the properties/metadata from our queue, let's use `get_queue_properties()`.

In [26]:
queue_client.get_queue_properties()

{'name': 'bens-queue-1', 'metadata': {}, 'approximate_message_count': 4}

To retrieve more than one message off the front of the queue, you can use `receive_messages()`.

In [27]:
messages = queue_client.receive_messages()

Let's print out all the messages we have so far.

In [28]:
for msg in messages:
    print(msg.content)

My first message
My second message
My second message
Was this my first message?


Let's now delete all of the messages in our queue. First, we'll retrieve all the messages.

In [32]:
messages = queue_client.receive_messages()

To delete the message, you need the message object or ID, so we'll loop through and print them first.

In [33]:
for msg in messages:
    print(msg.id)
    queue_client.delete_message(msg)

5b0b3330-8e67-416a-9e5a-5ba9cea9f79b
31ac7778-d5c9-4c89-81c7-fac644a4d20c
7f166b84-da9b-4c1c-8256-56969515acb4
2f8cbc3a-ae75-406c-acb1-7a084c1fc07a


Now that our queue is empty, let's delete the queue.

In [34]:
queue_client.delete_queue()